In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
torch.cuda.is_available()

In [32]:
import torch.nn as nn

In [35]:
x = torch.rand(1,10)

In [36]:
x

tensor([[0.3472, 0.4268, 0.2569, 0.5298, 0.6082, 0.8769, 0.6337, 0.6792, 0.1333,
         0.0773]])

In [46]:
a = torch.floor(x).type(torch.bool)

In [66]:
c = torch.tensor([False, True, False])
b = x._indices()[:,c]

In [76]:
i = torch.LongTensor([[1, 2, 3], [4, 5, 6]])

tensor([[1, 2, 3],
        [4, 5, 6]])

In [77]:
torch.transpose(i, 0,1)

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [39]:
import tensorflow as tf

In [87]:
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[3, 2])
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[2, 3])
c = tf.matmul(a, b)

In [88]:
c

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [89]:
tf.reshape(c, [-1])

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 27,  30,  33,  61,  68,  75,  95, 106, 117], dtype=int32)>

In [40]:
num_nonzero_elems = 5
noise_shape = [num_nonzero_elems]

In [41]:
temp = tf.random.uniform(noise_shape)

In [43]:
tf.floor(temp)

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
keep_prob = 0.7
random_tensor = keep_prob
temp = tf.random.uniform(noise_shape)
print(temp)
random_tensor += temp
print(random_tensor)

In [ ]:
random_tensor

In [28]:
import pandas as pd
import numpy as np

In [29]:
dataFile = pd.read_csv('../data/full_context_PeerRead.csv')

In [30]:
dataFile.head(100)


,right_citated_text,left_citated_text,source_abstract,source_author,source_id,source_title,source_venue,source_year,target_id,target_author,target_abstract,target_year,target_title,target_venue
0,andsyntactic parsing .Because RNNs make very f...,We conducted additional experiments on artific...,Deep Neural Networks (DNNs) are powerful model...,ilya sutskever;oriol vinyals;quoc v le,1409.3215v1,Sequence to Sequence Learning with Neural Netw...,NIPS,2014.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
1,.Because RNNs make very few domain-specific as...,We conducted additional experiments on artific...,Syntactic parsing is a fundamental problem in ...,oriol vinyals;lukasz kaiser;terry koo;slav pet...,1412.7449v1,Grammar as a Foreign Language,NIPS,2014.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
2,"; in a Pointer Network,the only way to generat...","Reproducibility. All code, data, and experimen...",We introduce a new neural architecture to lear...,oriol vinyals;meire fortunato;navdeep jaitly,1506.03134v1,Pointer Networks,NIPS,2015.0,1606.03622v1,robin jia;percy liang,Modeling crisp logical regularities is crucial...,2016,Data Recombination for Neural Semantic Parsing,ACL
3,". Recently, nsur . have shown superior perfor...","st like CWS and POS tagging, automatic prosody...",The recently introduced continuous Skip-gram m...,tomas mikolov;ilya sutskever;kai chen 0010;gre...,1310.4546v1,Distributed Representations of Words and Phras...,NIPS,2013.0,1511.00360v1,chuang ding;lei xie;jie yan;weini zhang;yang liu,Prosody affects the naturalness and intelligib...,2015,Automatic Prosody Prediction for Chinese Speec...,arxiv
4,model trained on the Google News dataset3.In a...,We begin by considering a document as the set ...,The recently introduced continuous Skip-gram m...,tomas mikolov;ilya sutskever;kai chen 0010;gre...,1310.4546v1,Distributed Representations of Words and Phras...,NIPS,2013.0,1705.10900v1,paul michel;abhilasha ravichander;shruti rijhwani,We investigate the pertinence of methods from ...,2017,Does the Geometry of Word Embeddings Help Docu...,arxiv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"to create a sentence embedding. Second, from e...",For each word in the sentence we calculate var...,In this paper we compare different types of re...,junyoung chung;caglar gulcehre;kyunghyun cho;y...,1412.3555v1,Empirical Evaluation of Gated Recurrent Neural...,arxiv,2014.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
96,for accelerating training. The network is opti...,For each Q-R pair we extract two sets of featu...,Training Deep Neural Networks is complicated b...,sergey ioffe;christian szegedy,1502.03167v1,Batch Normalization: Accelerating Deep Network...,ICML,2015.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
97,optimizer with learning rate of 0.001. We have...,For each Q-R pair we extract two sets of featu...,"We introduce Adam, an algorithm for first-orde...",diederik p kingma;jimmy ba,1412.6980v1,Adam: A Method for Stochastic Optimization,iclr,2014.0,1708.05582v1,sushant hiray;venkatesh duppada,This paper presents models for detecting agree...,2017,Agree to Disagree: Improving Disagreement Dete...,arxiv
98,", and convolutional neural networks have show...",Modeling textual or visual information with ve...,Deep Neural Networks (DNNs) are powerful model...,ilya sutskever;oriol vinyals;quoc v le,1409.3215v1,Sequence to Sequence Learning with Neural Netw...,NIPS,2014.0,1606.01847v1,akira fukui;dong huk park;daylen yang;anna roh...,Modeling textual or visual information with ve...,2016,Multimodal Compa

In [31]:
column = ['left_citated_text', 'right_citated_text', 'target_id', 'source_id', 'target_year','target_author', 'source_author']
frequency = 5

In [32]:
df = dataFile[column]
source_cut_data = df[['target_id', 'source_id']].drop_duplicates(subset=['target_id', 'source_id'])
source_cut = source_cut_data.source_id.value_counts()[(source_cut_data.source_id.value_counts() >= frequency)]

In [33]:
source_id = np.sort(source_cut.keys())
source_id
df = df.loc[df['source_id'].isin(source_id)]

In [103]:
source_id

array(['0705.4485v1', '0806.4686v1', '0812.4952v1', '0902.1284v1',
       '0902.2206v1', '0907.1815v1', '0911.5708v1', '0912.0071v1',
       '0912.3995v1', '1006.2588v1', '1010.3091v1', '1010.5511v1',
       '1103.0398v1', '1104.4803v1', '1105.1033v1', '1105.5379v1',
       '1106.2436v1', '1106.4574v1', '1107.4080v1', '1107.4557v1',
       '1109.3701v1', '1109.3843v1', '1109.5647v1', '1109.6841v1',
       '1112.6209v1', '1201.0292v1', '1201.0490v1', '1202.6258v1',
       '1204.0136v1', '1204.2742v1', '1204.3514v1', '1204.3968v1',
       '1204.6703v1', '1205.2661v1', '1206.1106v1', '1206.1270v1',
       '1206.2944v1', '1206.3255v1', '1206.4657v1', '1206.4683v1',
       '1206.6230v1', '1206.6380v1', '1206.6389v1', '1206.6392v1',
       '1206.6398v1', '1206.6400v1', '1206.6417v1', '1206.6418v1',
       '1206.6423v1', '1206.6426v1', '1206.6430v1', '1206.6448v1',
       '1206.6487v1', '1207.4404v1', '1207.4747v1', '1209.3352v1',
       '1210.5644v1', '1211.3966v1', '1211.5063v1', '1211.5590

In [101]:
df

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author
0,We conducted additional experiments on artific...,andsyntactic parsing .Because RNNs make very f...,1606.03622v1,1409.3215v1,2016,robin jia;percy liang,ilya sutskever;oriol vinyals;quoc v le
1,We conducted additional experiments on artific...,.Because RNNs make very few domain-specific as...,1606.03622v1,1412.7449v1,2016,robin jia;percy liang,oriol vinyals;lukasz kaiser;terry koo;slav pet...
2,"Reproducibility. All code, data, and experimen...","; in a Pointer Network,the only way to generat...",1606.03622v1,1506.03134v1,2016,robin jia;percy liang,oriol vinyals;meire fortunato;navdeep jaitly
3,"st like CWS and POS tagging, automatic prosody...",". Recently, nsur . have shown superior perfor...",1511.00360v1,1310.4546v1,2015,chuang ding;lei xie;jie yan;weini zhang;yang liu,tomas mikolov;ilya sutskever;kai chen 0010;gre...
4,We begin by considering a document as the set ...,model trained on the Google News dataset3.In a...,1705.10900v1,1310.4546v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,tomas mikolov;ilya sutskever;kai chen 0010;gre...
...,...,...,...,...,...,...,...
16664,The final argument tells LLAMA whether low pe...,". For each instance, 36 features weremeasured....",1306.1031v1,1306.5606v1,2013,lars kotthoff,barry hurley;lars kotthoff;yuri malitsky;barry...
16665,"This approach works reasonably well, but does ...",on the POS sequences in PTB trainingset.The re...,1610.03946v1,1301.3781v1,2016,jessica ficler;yoav goldberg,tomas mikolov;kai chen;greg corrado;jeffrey dean
16666,In addition to the symmetry and replacement si...,"and nia Treebank7.When evaluating on the PTB, ...",1610.03946v1,1606.02529v1,2016,jessica ficler;yoav goldberg,jessica ficler;yoav goldberg
16667,Fig. 10 shows histograms of the execution time...,", but none of these methods can be applied dir...",1708.04033v1,1603.02199v1,2017,tadanobu inoue;giovanni de magistris;asim muna...,sergey levine;peter pastor;alex krizhevsky;dei...


In [34]:
df['#1 String'] = df['left_citated_text'].str[-128:]
df['#2 String'] = df['right_citated_text'].str[:128]
df

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author,#1 String,#2 String
0,We conducted additional experiments on artific...,andsyntactic parsing .Because RNNs make very f...,1606.03622v1,1409.3215v1,2016,robin jia;percy liang,ilya sutskever;oriol vinyals;quoc v le,"le, recurrent neural networks have made swift...",andsyntactic parsing .Because RNNs make very f...
1,We conducted additional experiments on artific...,.Because RNNs make very few domain-specific as...,1606.03622v1,1412.7449v1,2016,robin jia;percy liang,oriol vinyals;lukasz kaiser;terry koo;slav pet...,networks have made swift inroads intomany str...,.Because RNNs make very few domain-specific as...
2,"Reproducibility. All code, data, and experimen...","; in a Pointer Network,the only way to generat...",1606.03622v1,1506.03134v1,2016,robin jia;percy liang,oriol vinyals;meire fortunato;navdeep jaitly,n-based copying can be seen as acombination of...,"; in a Pointer Network,the only way to generat..."
3,"st like CWS and POS tagging, automatic prosody...",". Recently, nsur . have shown superior perfor...",1511.00360v1,1310.4546v1,2015,chuang ding;lei xie;jie yan;weini zhang;yang liu,tomas mikolov;ilya sutskever;kai chen 0010;gre...,sing neural networks from raw text in a fully ...,". Recently, nsur . have shown superior perfor..."
4,We begin by considering a document as the set ...,model trained on the Google News dataset3.In a...,1705.10900v1,1310.4546v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,tomas mikolov;ilya sutskever;kai chen 0010;gre...,s their usefulness for real-world tasks.As a f...,model trained on the Google News dataset3.In a...
...,...,...,...,...,...,...,...,...,...
16662,"With human annotation of data, significant int...",proposed a yesian EM framework for continuous-...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,ion of each expert annotator and the underlyin...,proposed a yesian EM framework for continuous-...
16663,An effective probabilistic approach to aggrega...,. as is defined as the inverse of accuracy: It...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,reduce annotator inter- and intra-variability....,. as is defined as the inverse of accuracy: It...
16665,"This approach works reasonably well, but does ...",on the POS sequences in PTB trainingset.The re...,1610.03946v1,1301.3781v1,2016,jessica ficler;yoav goldberg,tomas mikolov;kai chen;greg corrado;jeffrey dean,"ell in the CKY chart.3In both approaches,the P...",on the POS sequences in PTB trainingset.The re...
16667,Fig. 10 shows histograms of the execution time...,", but none of these methods can be applied dir...",1708.04033v1,1603.02199v1,2017,tadanobu inoue;giovanni de magistris;asim muna...,sergey levine;peter pastor;alex krizhevsky;dei...,l concept is shown in Fig. 1.Recent studies ha...,", but none of these methods can be applied dir..."


In [35]:
left_citated_text  = df['#1 String'].values.tolist()
right_citated_text = df['#2 String'].values.tolist()
total_citated_text = list(set(left_citated_text + right_citated_text))
citated_voca = {}
left_citated_id = []
right_citated_id = []
for i, v in enumerate(total_citated_text):
    citated_voca[v] = i
citated_voca

{'ameters for the text representation model. We take Jsemi as the loss function, and train neural networks with the Adam algorithm': 0,
 'ck-box attacks without a substitute model , based on the principle that adversarial examples can transfer among different models': 1,
 'vel models .Outside of language modeling, improvements have been reported on part-of-speech tagging  andnamed entity recognition': 2,
 ' models such as Deep Convolutional GAN  , undary Equilibrium GAN  , and the combination of DCGAN with a Variational Autoencoder ': 3,
 'by leveraging semantic denotations during structural search.Third, to train from weak supervision and directly maximize the expe': 4,
 'introduced a simplified version of LSTMs called Gated Recurrent Units  with has one less gate, and consequently fewer parameters': 5,
 '. Architectures employed for NLP applications differ in that they typically involve temporal ratherthan spatial convolutions.t C': 6,
 'explore the magnitude-based pruning in conjunc

In [36]:
for l, r in zip(df['#1 String'], df['#2 String']):
        left_citated_id.append(citated_voca[l])
        right_citated_id.append(citated_voca[r])

In [37]:
df['#1 ID'] = left_citated_id
df['#2 ID'] = right_citated_id

In [38]:
df

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author,#1 String,#2 String,#1 ID,#2 ID
0,We conducted additional experiments on artific...,andsyntactic parsing .Because RNNs make very f...,1606.03622v1,1409.3215v1,2016,robin jia;percy liang,ilya sutskever;oriol vinyals;quoc v le,"le, recurrent neural networks have made swift...",andsyntactic parsing .Because RNNs make very f...,24142,12541
1,We conducted additional experiments on artific...,.Because RNNs make very few domain-specific as...,1606.03622v1,1412.7449v1,2016,robin jia;percy liang,oriol vinyals;lukasz kaiser;terry koo;slav pet...,networks have made swift inroads intomany str...,.Because RNNs make very few domain-specific as...,23803,4098
2,"Reproducibility. All code, data, and experimen...","; in a Pointer Network,the only way to generat...",1606.03622v1,1506.03134v1,2016,robin jia;percy liang,oriol vinyals;meire fortunato;navdeep jaitly,n-based copying can be seen as acombination of...,"; in a Pointer Network,the only way to generat...",11893,448
3,"st like CWS and POS tagging, automatic prosody...",". Recently, nsur . have shown superior perfor...",1511.00360v1,1310.4546v1,2015,chuang ding;lei xie;jie yan;weini zhang;yang liu,tomas mikolov;ilya sutskever;kai chen 0010;gre...,sing neural networks from raw text in a fully ...,". Recently, nsur . have shown superior perfor...",23709,15528
4,We begin by considering a document as the set ...,model trained on the Google News dataset3.In a...,1705.10900v1,1310.4546v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,tomas mikolov;ilya sutskever;kai chen 0010;gre...,s their usefulness for real-world tasks.As a f...,model trained on the Google News dataset3.In a...,287,5463
...,...,...,...,...,...,...,...,...,...,...,...
16662,"With human annotation of data, significant int...",proposed a yesian EM framework for continuous-...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,ion of each expert annotator and the underlyin...,proposed a yesian EM framework for continuous-...,5633,8530
16663,An effective probabilistic approach to aggrega...,. as is defined as the inverse of accuracy: It...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,reduce annotator inter- and intra-variability....,. as is defined as the inverse of accuracy: It...,2682,14024
16665,"This approach works reasonably well, but does ...",on the POS sequences in PTB trainingset.The re...,1610.03946v1,1301.3781v1,2016,jessica ficler;yoav goldberg,tomas mikolov;kai chen;greg corrado;jeffrey dean,"ell in the CKY chart.3In both approaches,the P...",on the POS sequences in PTB trainingset.The re...,20866,6656
16667,Fig. 10 shows histograms of the execution time...,", but none of these methods can be applied dir...",1708.04033v1,1603.02199v1,2017,tadanobu inoue;giovanni de magistris;asim muna...,sergey levine;peter pastor;alex krizhevsky;dei...,l concept is shown in Fig. 1.Recent studies ha...,", but none of these methods can be applied dir...",19226,15790


In [39]:
year = 2017
train_idx = df['target_year'][df['target_year'] < year].index
test_idx = df['target_year'][df['target_year'] >= year].index
train_df = df.loc[train_idx]
test_df = df.loc[test_idx]

In [40]:
test_df

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author,#1 String,#2 String,#1 ID,#2 ID
4,We begin by considering a document as the set ...,model trained on the Google News dataset3.In a...,1705.10900v1,1310.4546v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,tomas mikolov;ilya sutskever;kai chen 0010;gre...,s their usefulness for real-world tasks.As a f...,model trained on the Google News dataset3.In a...,287,5463
5,"With GMM and Aw2v+Ph, the F1-Score of clusteri...",". We observe that by themselves, Ph embeddings...",1705.10900v1,1408.5882v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,yoon kim,"presented in Table 1. For comparison, we provi...",". We observe that by themselves, Ph embeddings...",1464,14906
6,We evaluate our method on the CSP dataset8. Th...,. We observe that Ph embeddings perform poorly...,1705.10900v1,1405.4053v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,quoc v le;tomas mikolov,e word vectors in that sentence. The results a...,. We observe that Ph embeddings perform poorly...,1563,19315
7,We perform document-level binary sentiment cla...,or purely word vector based methods . .cument...,1705.10900v1,1405.4053v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,quoc v le;tomas mikolov,d recent attention owing to their utility in s...,or purely word vector based methods . .cument...,22201,19816
8,In the lack of sufficient health status inform...,. Two most popular approaches of DGM rely on v...,1709.00845v1,1406.5298v1,2017,andre s yoon;taehoon lee;yongsub lim;deokwoo j...,diederik p kingma;shakir mohamed;danilo jimene...,enerative models have recently achieved state...,. Two most popular approaches of DGM rely on v...,6100,13092
...,...,...,...,...,...,...,...,...,...,...,...
16649,We now demonstrate cascading modern CNN archit...,", as a reference and construct a similar archi...",1703.08961v1,1605.07146v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey zagoruyko;nikos komodakis,ication task. re we consider cascading the sca...,", as a reference and construct a similar archi...",1765,17047
16650,For the scattering transform we used J=2 which...,. Specifically we modify the WRN of 16 layers ...,1703.08961v1,1605.07146v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey zagoruyko;nikos komodakis,"on CIFAR-10, all based on end-to-end learned C...",. Specifically we modify the WRN of 16 layers ...,22417,14237
16651,We now consider the popular CIFAR-10 dataset c...,. Table 3 reports the accuracy in the unsuperv...,1703.08961v1,1502.03167v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey ioffe;christian szegedy,on in both case. We utilize batch normalizatio...,. Table 3 reports the accuracy in the unsuperv...,22240,17408
16667,Fig. 10 shows histograms of the execution time...,", but none of these methods can be applied dir...",1708.04033v1,1603.02199v1,2017,tadanobu inoue;giovanni de magistris;asim muna...,sergey levine;peter pastor;alex krizhevsky;dei...,l concept is shown in Fig. 1.Recent studies ha...,", but none of these methods can be applied dir...",19226,15790


In [41]:
bert_column = ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String', 'target_id', 'source_author']

In [42]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit_transform(df['source_id'].values)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
lb.get_params()

{'neg_label': 0, 'pos_label': 1, 'sparse_output': False}

In [44]:
def convert_argmax(df, lb):

    y = df['source_id'].values
    y = lb.transform(y)
    y = np.argmax(y, axis=1)
    df['Quality'] = y

    return df

In [45]:
train_df1 = convert_argmax(train_df, lb)

In [46]:
train_df1

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author,#1 String,#2 String,#1 ID,#2 ID,Quality
0,We conducted additional experiments on artific...,andsyntactic parsing .Because RNNs make very f...,1606.03622v1,1409.3215v1,2016,robin jia;percy liang,ilya sutskever;oriol vinyals;quoc v le,"le, recurrent neural networks have made swift...",andsyntactic parsing .Because RNNs make very f...,24142,12541,139
1,We conducted additional experiments on artific...,.Because RNNs make very few domain-specific as...,1606.03622v1,1412.7449v1,2016,robin jia;percy liang,oriol vinyals;lukasz kaiser;terry koo;slav pet...,networks have made swift inroads intomany str...,.Because RNNs make very few domain-specific as...,23803,4098,174
2,"Reproducibility. All code, data, and experimen...","; in a Pointer Network,the only way to generat...",1606.03622v1,1506.03134v1,2016,robin jia;percy liang,oriol vinyals;meire fortunato;navdeep jaitly,n-based copying can be seen as acombination of...,"; in a Pointer Network,the only way to generat...",11893,448,232
3,"st like CWS and POS tagging, automatic prosody...",". Recently, nsur . have shown superior perfor...",1511.00360v1,1310.4546v1,2015,chuang ding;lei xie;jie yan;weini zhang;yang liu,tomas mikolov;ilya sutskever;kai chen 0010;gre...,sing neural networks from raw text in a fully ...,". Recently, nsur . have shown superior perfor...",23709,15528,90
13,stillation has been traditionally applied to n...,usedcompression to transfer knowledge from a d...,1511.06295v1,1312.6184v1,2015,andrei a rusu;sergio gomez colmenarejo;caglar ...,jimmy ba;rich caruana,"by cila . , whoproposed it as a means of comp...",usedcompression to transfer knowledge from a d...,10542,14649,102
...,...,...,...,...,...,...,...,...,...,...,...,...
16660,An important inspiration for the proposed fram...,. Two important differenceswith the approach p...,1502.04156v1,1406.2751v1,2015,yoshua bengio;dong-hyun lee;jorg bornschein;th...,"j\""org bornschein;yoshua bengio",lgorithm and which finds very interesting ins...,. Two important differenceswith the approach p...,24106,207,127
16661,The proposal made here also owes a lot to the ...,"and generativeadversarial networks ( ., 2014)...",1502.04156v1,1306.1091v1,2015,yoshua bengio;dong-hyun lee;jorg bornschein;th...,yoshua bengio;eric laufer;guillaume alain;jaso...,about the same or better as was obtained for c...,"and generativeadversarial networks ( ., 2014)...",23578,10752,80
16662,"With human annotation of data, significant int...",proposed a yesian EM framework for continuous-...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,ion of each expert annotator and the underlyin...,proposed a yesian EM framework for continuous-...,5633,8530,319
16663,An effective probabilistic approach to aggrega...,. as is defined as the inverse of accuracy: It...,1503.06619v1,1512.02393v1,2015,tingting zhu;nic dunkley;joachim behar;david a...,changbo zhu;huan xu;shuicheng yan,reduce annotator inter- and intra-variability....,. as is defined as the inverse of accuracy: It...,2682,14024,319


In [48]:
anArray = train_df1['Quality'].values

In [116]:
test_df1 = convert_argmax(test_df, lb)

In [117]:
test_df1

,left_citated_text,right_citated_text,target_id,source_id,target_year,target_author,source_author,#1 String,#2 String,#1 ID,#2 ID,Quality
4,We begin by considering a document as the set ...,model trained on the Google News dataset3.In a...,1705.10900v1,1310.4546v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,tomas mikolov;ilya sutskever;kai chen 0010;gre...,s their usefulness for real-world tasks.As a f...,model trained on the Google News dataset3.In a...,23427,10153,90
5,"With GMM and Aw2v+Ph, the F1-Score of clusteri...",". We observe that by themselves, Ph embeddings...",1705.10900v1,1408.5882v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,yoon kim,"presented in Table 1. For comparison, we provi...",". We observe that by themselves, Ph embeddings...",13225,377,136
6,We evaluate our method on the CSP dataset8. Th...,. We observe that Ph embeddings perform poorly...,1705.10900v1,1405.4053v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,quoc v le;tomas mikolov,e word vectors in that sentence. The results a...,. We observe that Ph embeddings perform poorly...,15416,1612,120
7,We perform document-level binary sentiment cla...,or purely word vector based methods . .cument...,1705.10900v1,1405.4053v1,2017,paul michel;abhilasha ravichander;shruti rijhwani,quoc v le;tomas mikolov,d recent attention owing to their utility in s...,or purely word vector based methods . .cument...,12091,22025,120
8,In the lack of sufficient health status inform...,. Two most popular approaches of DGM rely on v...,1709.00845v1,1406.5298v1,2017,andre s yoon;taehoon lee;yongsub lim;deokwoo j...,diederik p kingma;shakir mohamed;danilo jimene...,enerative models have recently achieved state...,. Two most popular approaches of DGM rely on v...,9400,16385,130
...,...,...,...,...,...,...,...,...,...,...,...,...
16649,We now demonstrate cascading modern CNN archit...,", as a reference and construct a similar archi...",1703.08961v1,1605.07146v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey zagoruyko;nikos komodakis,ication task. re we consider cascading the sca...,", as a reference and construct a similar archi...",11129,1356,397
16650,For the scattering transform we used J=2 which...,. Specifically we modify the WRN of 16 layers ...,1703.08961v1,1605.07146v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey zagoruyko;nikos komodakis,"on CIFAR-10, all based on end-to-end learned C...",. Specifically we modify the WRN of 16 layers ...,13604,7143,397
16651,We now consider the popular CIFAR-10 dataset c...,. Table 3 reports the accuracy in the unsuperv...,1703.08961v1,1502.03167v1,2017,edouard oyallon;eugene belilovsky;sergey zagor...,sergey ioffe;christian szegedy,on in both case. We utilize batch normalizatio...,. Table 3 reports the accuracy in the unsuperv...,2229,1423,185
16667,Fig. 10 shows histograms of the execution time...,", but none of these methods can be applied dir...",1708.04033v1,1603.02199v1,2017,tadanobu inoue;giovanni de magistris;asim muna...,sergey levine;peter pastor;alex krizhevsky;dei...,l concept is shown in Fig. 1.Recent studies ha...,", but none of these methods can be applied dir...",20485,21074,357


In [118]:
train_df, test_df = train_df1[bert_column], test_df1[bert_column]

In [119]:
test_df

,Quality,#1 ID,#2 ID,#1 String,#2 String,target_id,source_author
4,90,23427,10153,s their usefulness for real-world tasks.As a f...,model trained on the Google News dataset3.In a...,1705.10900v1,tomas mikolov;ilya sutskever;kai chen 0010;gre...
5,136,13225,377,"presented in Table 1. For comparison, we provi...",". We observe that by themselves, Ph embeddings...",1705.10900v1,yoon kim
6,120,15416,1612,e word vectors in that sentence. The results a...,. We observe that Ph embeddings perform poorly...,1705.10900v1,quoc v le;tomas mikolov
7,120,12091,22025,d recent attention owing to their utility in s...,or purely word vector based methods . .cument...,1705.10900v1,quoc v le;tomas mikolov
8,130,9400,16385,enerative models have recently achieved state...,. Two most popular approaches of DGM rely on v...,1709.00845v1,diederik p kingma;shakir mohamed;danilo jimene...
...,...,...,...,...,...,...,...
16649,397,11129,1356,ication task. re we consider cascading the sca...,", as a reference and construct a similar archi...",1703.08961v1,sergey zagoruyko;nikos komodakis
16650,397,13604,7143,"on CIFAR-10, all based on end-to-end learned C...",. Specifically we modify the WRN of 16 layers ...,1703.08961v1,sergey zagoruyko;nikos komodakis
16651,185,2229,1423,on in both case. We utilize batch normalizatio...,. Table 3 reports the accuracy in the unsuperv...,1703.08961v1,sergey ioffe;christian szegedy
16667,357,20485,21074,l concept is shown in Fig. 1.Recent studies ha...,", but none of these methods can be applied dir...",1708.04033v1,sergey levine;peter pastor;alex krizhevsky;dei...


In [120]:
train_df

,Quality,#1 ID,#2 ID,#1 String,#2 String,target_id,source_author
0,139,8103,7582,"le, recurrent neural networks have made swift...",andsyntactic parsing .Because RNNs make very f...,1606.03622v1,ilya sutskever;oriol vinyals;quoc v le
1,174,818,21392,networks have made swift inroads intomany str...,.Because RNNs make very few domain-specific as...,1606.03622v1,oriol vinyals;lukasz kaiser;terry koo;slav pet...
2,232,10444,2237,n-based copying can be seen as acombination of...,"; in a Pointer Network,the only way to generat...",1606.03622v1,oriol vinyals;meire fortunato;navdeep jaitly
3,90,5440,14282,sing neural networks from raw text in a fully ...,". Recently, nsur . have shown superior perfor...",1511.00360v1,tomas mikolov;ilya sutskever;kai chen 0010;gre...
13,102,7301,3444,"by cila . , whoproposed it as a means of comp...",usedcompression to transfer knowledge from a d...,1511.06295v1,jimmy ba;rich caruana
...,...,...,...,...,...,...,...
16660,127,19965,15181,lgorithm and which finds very interesting ins...,. Two important differenceswith the approach p...,1502.04156v1,"j\""org bornschein;yoshua bengio"
16661,80,1787,20657,about the same or better as was obtained for c...,"and generativeadversarial networks ( ., 2014)...",1502.04156v1,yoshua bengio;eric laufer;guillaume alain;jaso...
16662,319,6892,723,ion of each expert annotator and the underlyin...,proposed a yesian EM framework for continuous-...,1503.06619v1,changbo zhu;huan xu;shuicheng yan
16663,319,20408,15313,reduce annotator inter- and intra-variability....,. as is defined as the inverse of accuracy: It...,1503.06619v1,changbo zhu;huan xu;shuicheng yan


In [ ]:
train_df.loc[train_df['#1 ID'] == 14050]

In [ ]:
temp_df = test_df

In [ ]:
temp_df.reset_index()

In [ ]:
multi_label_info = []
for i in temp_df.groupby(['#1 ID', '#2 ID']):
    instance_label = {}
    instance_label['Quality'] = i[1]['Quality'].values
    instance_label['index'] = i[1]['Quality'].index.values
    multi_label_info.append(instance_label)

In [ ]:
for i in range (0,len(multi_label_info)):
    if multi_label_info[i]['index'].any() == 0:
        print(multi_label_info[i])
        print(i)

In [ ]:
multi_label_info[0]['index']

In [ ]:
import tensorflow as tf
print("Num GPUs Available: HI ", len(tf.config.experimental.list_physical_devices('GPU')))

### Preprocessing

In [1]:
import pandas as pd
import transformers
import torch
import tqdm

In [2]:
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [3]:
a = tokenizer.encode_plus("Hi. How are you?","Hi. I am fine and how are you?", add_special_tokens = True, max_length = 100, padding="max_length")

In [254]:
b = tokenizer.encode_plus("Hi. I am fine and how are you?", None, add_special_tokens = True, max_length = 100, padding=True)

In [5]:
len(a["input_ids"])

100

In [6]:
a["input_ids"]

[101,
 7632,
 1012,
 2129,
 2024,
 2017,
 1029,
 102,
 7632,
 1012,
 1045,
 2572,
 2986,
 1998,
 2129,
 2024,
 2017,
 1029,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [10]:
tokenizer.cls_token, tokenizer.cls_token_id

('[CLS]', 101)

In [15]:
context = "You are using pip version.Just like a watched pot never boils"
print(" ".join(context.split()))

You are using pip version.Just like a watched pot never boils


In [20]:
dataFile["right_citated_text"].values[0]

"andsyntactic parsing .Because RNNs make very few domain-specific assumptions,they have the potential to succeed at a wide variety of taskswith minimal feature engineering.wever, this flexibility also puts RNNs at a disadvantage compared tostandard semantic parsers, which can generalize naturallyby leveraging their built-in awareness of logical compositionality.In this paper, we introduce data recombination,a generic framework for declaratively injecting prior knowledgeinto a domain-general structured prediction model.In data recombination, prior knowledge about a taskis used to build a high-precision generative modelthat expands the empirical distributionby allowing fragments of different examples to be combined inparticular ways.mples from this generative model are then used to train a domain-general model.In the case of semantic parsing, we construct a generative modelby inducing a synchronous context-free grammar ,creating new examples such as those shown in Figure 1;our domain-gen

In [ ]:
def get_gcn_data(train_df, test_df, save_dir):
    with open('{}/gcn_pretrain.pkl'.format(save_dir), 'rb') as f:
        embedding = pickle.load(f)
        node2id = pickle.load(f)
    gcn_train = np.array([[node2id[i]] for i in train_df['query_id'].values])
    gcn_test = np.array([[node2id[i]] for i in test_df['query_id'].values])
    return gcn_train, gcn_test, embedding

In [92]:
embedding = None
node2id = None
with open('/mnt/e/MS/2-1B/AP-FIR/bert-gcn/pre_train/gcn/PeerRead/PeerRead_gcn_pretrain_author.pkl', 'rb') as f:
        embedding = pickle.load(f)
        node2id = pickle.load(f)

In [91]:
import pickle

In [95]:
len(embedding[0])

768

In [130]:
train_df['source_author'].values[0].split(';')[-1]

'quoc v le'

In [131]:
arr = train_df['source_author'].values

In [189]:
cnt = 0
for i in arr:
    cnt += 1
    if i.split(';')[-1] == 'afroze ibrahim baqapuri':
        print('Teri maa ki ', cnt)
    

Teri maa ki  1111
Teri maa ki  1112
Teri maa ki  1264
Teri maa ki  1265
Teri maa ki  1266
Teri maa ki  1267
Teri maa ki  1358
Teri maa ki  1359
Teri maa ki  1360
Teri maa ki  1361
Teri maa ki  2168
Teri maa ki  2169
Teri maa ki  2698
Teri maa ki  2840
Teri maa ki  2850
Teri maa ki  3943
Teri maa ki  4738
Teri maa ki  5316
Teri maa ki  5317
Teri maa ki  5318
Teri maa ki  5430
Teri maa ki  6831


In [185]:
node2id['afroze ibrahim baqapuri']

KeyError: 'afroze ibrahim baqapuri'

In [139]:
embedding[3564]

array([-4.75661084e-02, -2.11767226e-01, -7.69936517e-02, -1.34367093e-01,
       -1.32247269e-01,  9.25423503e-02, -1.23237289e-01, -2.50031084e-01,
        1.83598384e-01, -1.87942892e-01,  1.05373263e-02, -3.16464901e-03,
        2.89688170e-01,  1.16193756e-01,  3.08152884e-02, -1.64103135e-01,
        3.85200605e-02,  1.93548799e-02,  1.04115993e-01,  1.59260869e-01,
        3.75042222e-02, -4.13777083e-01, -1.40298188e-01, -7.97704756e-02,
        1.69954151e-02,  9.29089710e-02,  5.71189970e-02, -9.63231102e-02,
        2.62236208e-01,  2.40637809e-02,  1.57377809e-01, -1.58616498e-01,
       -1.79514438e-01,  1.06614847e-02, -4.85762879e-02,  8.04453343e-02,
       -7.72103742e-02, -1.38636418e-02, -5.83091676e-02,  3.78895551e-03,
       -1.35671839e-01,  8.73243362e-02, -8.78408253e-02,  7.90649429e-02,
        3.46747875e-01,  2.61593997e-01,  1.78148653e-02, -1.58512473e-01,
        2.26705939e-01, -1.97524950e-01,  2.39470914e-01, -1.20129056e-01,
        3.78371663e-02,  

In [208]:
gcn_train = np.zeros(shape=(1,768))

In [209]:
b = np.append(gcn_train, [embedding[node2id[arr[0].split(';')[-1]]]], axis=0)
b = np.append(b, [embedding[node2id[arr[1].split(';')[-1]]]], axis=0)
b = np.append(b, [embedding[node2id[arr[2].split(';')[-1]]]], axis=0)

In [211]:
len(b)

4

In [191]:
node2id['afroze ibrahim baqapuri'] 

KeyError: 'afroze ibrahim baqapuri'

In [197]:
x = zip([1,2], [9,10], [])

In [198]:
list(x)

[]

In [212]:
loss = nn.CrossEntropyLoss()

In [213]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [216]:
output

tensor(2.2013, grad_fn=<NllLossBackward>)

In [215]:
output.backward()

In [218]:
target.shape[0]

3

In [245]:
targetMatrix = np.zeros(shape=(10, 3))
targets = np.array([0,2,2,1,0,2,1,1,0,0])

In [247]:
for i in range(0,targets.shape[0]):
    print(i, targets[[i]])
    targetMatrix[i,targets[[i]]] = 1

0 [0]
1 [2]
2 [2]
3 [1]
4 [0]
5 [2]
6 [1]
7 [1]
8 [0]
9 [0]


In [248]:
targetMatrix

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [239]:
targets

array([[0, 2, 2, 1, 0, 2, 1, 1, 0, 0]])

In [271]:
a = ('MM', ['hi', 'bye'], 'MM')

In [272]:
type(a)

tuple

In [273]:
a

('MM', ['hi', 'bye'], 'MM')

In [274]:
x = torch.randn(10, 5)
y = torch.randint(5, (10,))

In [275]:
x

tensor([[-0.6558, -0.6311, -1.1523, -0.4526, -0.1082],
        [-0.0242,  0.2569,  1.0637, -1.3420,  0.7725],
        [-1.3376,  0.0537, -0.2791, -1.9397,  0.3444],
        [-0.7768, -0.5065,  0.4023,  0.4000,  1.2417],
        [ 0.8871,  0.8222,  1.1341,  0.0854, -0.6681],
        [ 1.4926, -1.0424, -0.1974, -1.4365,  2.7164],
        [ 0.4590,  1.7561, -0.7182, -0.6271, -0.7176],
        [-0.5132,  1.5448,  1.0711,  0.4586,  0.7220],
        [-0.3532,  0.9306,  0.5386, -1.3265, -1.6723],
        [ 1.5191,  0.5804,  0.7835,  1.4071,  0.4309]])

In [279]:
y

tensor([4, 4, 1, 2, 3, 0, 3, 4, 3, 1])

In [280]:
loss = nn.CrossEntropyLoss()(x, y)

In [281]:
loss

tensor(1.8631)

In [10]:
import torch
outputs = torch.randn(10,10)

In [15]:
outputs.cpu().detach().numpy()

array([[ 1.5141089 ,  0.21601109,  0.3352844 , -0.47060046,  0.52831715,
         0.8986401 , -1.3377849 , -0.21681648,  0.3495095 ,  0.04451937],
       [ 0.82896525, -0.43477282, -0.6625765 , -1.1890138 ,  1.5161456 ,
         0.7036071 ,  0.09192441,  0.15651953,  0.834813  ,  0.8876868 ],
       [-1.6549444 ,  0.4601838 , -0.8080143 , -1.2026032 , -0.7485504 ,
        -0.01099867, -0.15947303,  1.3419424 , -0.48343274, -1.5495147 ],
       [-0.38402694,  0.20689583,  1.091941  ,  0.8155733 , -0.90273005,
         0.06526163,  1.5922868 , -0.741216  , -0.12303007,  0.14203379],
       [ 0.80925745, -0.8369664 ,  0.69213337,  0.41236097,  0.049534  ,
        -1.89061   , -1.6556381 , -0.33929276, -0.18388875,  0.22856106],
       [ 0.87471455,  0.5971452 ,  1.2274413 ,  0.62335396,  0.16428806,
         0.7154189 , -0.5671112 , -1.0328497 ,  0.64456457,  0.32417497],
       [ 0.35605755,  1.5302416 ,  0.04111671, -0.32558602, -1.2731838 ,
        -0.6747714 ,  1.0610003 ,  1.7816477 

In [17]:
temp = outputs.cpu().detach().numpy().tolist()

In [19]:
import scipy.stats as ss

In [23]:
ss.rankdata(temp[0])

array([10.,  5.,  6.,  2.,  8.,  9.,  1.,  3.,  7.,  4.])

In [26]:
strr = 'Ashish'
final = 'Hi '+strr

In [27]:
final

'Hi Ashish'

In [56]:
for i in range(0,len(anArray)):
    if (anArray[i]).size > 1:
        print(anArray[i])

In [77]:
import numpy as np
from sklearn.metrics import average_precision_score
y_true = np.array([[0,0,0,1], [1,0,0,0], [0,0,1,0], [0,1,0,0]])
y_scores = np.array([[0.2, 0.4, 0.1, 0.3],[0.2, 0.4, 0.1, 0.3],[0.2, 0.4, 0.1, 0.3],[0.2, 0.4, 0.1, 0.3]])
average_precision_score(y_true, y_scores)

0.25

In [78]:
y_scores.shape

(4, 4)

In [79]:
lol = np.zeros(y_scores.shape)

In [80]:
lol

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

SyntaxError: invalid syntax (<ipython-input-81-4ea62681798a>, line 1)